1. Gmailからの(指定の)受信メールを取得しデータフレームにする<br>
  [参考にしたサイト](https://di-acc2.com/system/rpa/3031/#index_id12)
2. データフレームを加工する

In [158]:
""" ライブラリ読込 """
import imapclient
from backports import ssl
from OpenSSL import SSL 
import pyzmail
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [159]:
import os

""" (要編集) 引数指定 """ 
#ログイン情報
my_mail = os.getenv("my_mail")
app_password = os.getenv("app_password")

#メール検索条件
FolderName = "INBOX"
Search_KWD = ["FROM", "1695kose@wits2003.jp"]

""" 関数定義 """
def Get_Mail(my_mail, app_password, FolderName, Search_KWD):
    
    """ ① IMAPサーバー接続 & SSL化 """
    context = ssl.SSLContext(SSL.TLSv1_2_METHOD)
    # context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
    imap = imapclient.IMAPClient("imap.gmail.com", ssl=True, ssl_context=context)
    
    """ ② IMAPログイン """
    imap.login(my_mail,app_password)
    
    
    """ ③ メール検索 """
    #対象の受信メールフォルダを指定
    imap.select_folder(FolderName, readonly=True)
    
    #検索キーワードを設定 & 検索キーワードに紐づくメールID検索
    KWD = imap.search(Search_KWD)
    
    #メールID→メール本文取得
    raw_message = imap.fetch(KWD,["BODY[]"])
    
    """  ④ 解析結果保存 """
    #検索結果保存用
    From_list     = []
    Cc_list       = []
    Bcc_list      = []
    Subject_list  = []
    Body_list     = []
    
    #メール解析
    for j in range(len(KWD)):
        #特定メール取得
        message = pyzmail.PyzMessage.factory(raw_message[KWD[j]][b"BODY[]"])

        #宛先取得
        From = message.get_addresses("from")
        From_list.append(From)

        Cc = message.get_addresses("cc")
        Cc_list.append(Cc)

        Bcc = message.get_addresses("bcc")
        Bcc_list.append(Bcc)

        #件名取得
        Subject = message.get_subject()
        Subject_list.append(Subject)

        #本文
        # "ignore"オプションをつけてdecodeすると、デコード不可な文字は無視して無理やりdecodeしてくれます。
        Body = message.text_part.get_payload().decode("iso-2022-jp", "ignore")
        Body_list.append(Body)


    #Pandas データフレーム
    table = pd.DataFrame({"From":From_list,
                          "Cc":Cc_list,
                          "Bcc":Bcc_list,
                          "件名":Subject_list,
                          "本文":Body_list,
                         })

    #IMAPログアウト
    #imap.logout()

    return table

In [160]:
# 関数実行
mail_df = Get_Mail(my_mail, app_password, FolderName, Search_KWD)

In [161]:
mail_df.head()

,From,Cc,Bcc,件名,本文
0,"[(1695kose@wits2003.jp, 1695kose@wits2003.jp)]",[],[],【受験生infoメール0923�（新着情報版）】【受験生infoメール0923�（過去送信版）】,（中３生の方へ送信）\r\n\r\nいつも大変お世話になっております，ITTOの松藤です。受...
1,"[(1695kose@wits2003.jp, 1695kose@wits2003.jp)]",[],[],【重要】保護者対象・入試説明会のお知らせ,（中３生の方へ送信）\r\nいつも大変お世話になっております，ITTOの松藤です。\r\n中...
2,"[(1695kose@wits2003.jp, 1695kose@wits2003.jp)]",[],[],【再送】入試説明会，最新の参加人数です。変更も受け付けております,※昨日送信したメールですが，参加人数を最新版に更新しております。\r\n※変更に応じていただ...
3,"[(1695kose@wits2003.jp, 1695kose@wits2003.jp)]",[],[],本年もよろしくお願いいたします！,（全保護者様へ送信）\r\n新年あけましておめでとうございます。今年もどうぞよろしくお願いい...
4,"[(1695kose@wits2003.jp, 1695kose@wits2003.jp)]",[],[],01月07日の授業日報につきまして,いつも大変お世話になりありがとうございます。\r\nITTO個別指導学院 佐賀巨勢校です。\...


# データフレームの加工

In [162]:
import numpy as np

In [163]:
# 授業日報だけのメールを取得する
report_df = mail_df[mail_df['件名'].str.contains('授業日報')]

In [164]:
# ●授業日　03月02日 (水)
# ●時間　20時00分から80分
# ●時間　20時00分から80分
# ●教科　

In [165]:
# 日付だけ、時間だけのカラムを作る
report_df['日付'] = report_df['本文'].apply(lambda x : x[x.find('●授業日')+5:x.find('●時間')-2])
report_df['時間'] = report_df['本文'].apply(lambda x : x[x.find('●時間')+4:x.find('●教科')-2])
# report_df['日時'] = report_df.apply(lambda x: f"{x['日付']} {x['時間']}", axis=1)
report_df.head()
# 時間だけのカラムを作る

/var/folders/x4/8hbz4flj6_bg96s9y3b69rb80000gn/T/ipykernel_64230/3406159213.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_df['日付'] = report_df['本文'].apply(lambda x : x[x.find('●授業日')+5:x.find('●時間')-2])
/var/folders/x4/8hbz4flj6_bg96s9y3b69rb80000gn/T/ipykernel_64230/3406159213.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_df['時間'] = report_df['本文'].apply(lambda x : x[x.find('●時間')+4:x.find('●教科')-2])


,From,Cc,Bcc,件名,本文,日付,時間
4,"[(1695kose@wits2003.jp, 1695kose@wits2003.jp)]",[],[],01月07日の授業日報につきまして,いつも大変お世話になりありがとうございます。\r\nITTO個別指導学院 佐賀巨勢校です。\...,01月07日 (金),13時00分から100分
5,"[(1695kose@wits2003.jp, 1695kose@wits2003.jp)]",[],[],01月07日の授業日報につきまして,いつも大変お世話になりありがとうございます。\r\nITTO個別指導学院 佐賀巨勢校です。\...,01月07日 (金),15時00分から100分
6,"[(1695kose@wits2003.jp, 1695kose@wits2003.jp)]",[],[],01月07日の授業日報につきまして,いつも大変お世話になりありがとうございます。\r\nITTO個別指導学院 佐賀巨勢校です。\...,01月07日 (金),20時00分から80分
9,"[(1695kose@wits2003.jp, 1695kose@wits2003.jp)]",[],[],01月15日の授業日報につきまして,いつも大変お世話になりありがとうございます。\r\nITTO個別指導学院 佐賀巨勢校です。\...,01月15日 (土),13時00分から100分
10,"[(1695kose@wits2003.jp, 1695kose@wits2003.jp)]",[],[],01月15日の授業日報につきまして,いつも大変お世話になりありがとうございます。\r\nITTO個別指導学院 佐賀巨勢校です。\...,01月15日 (土),18時00分から100分


In [166]:
# 講師だけのカラムを作る
report_df['講師'] = report_df['本文'].apply(lambda x : x[x.find('講師')+3:x.find('\r\n●授業日')])

/var/folders/x4/8hbz4flj6_bg96s9y3b69rb80000gn/T/ipykernel_64230/1344642912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_df['講師'] = report_df['本文'].apply(lambda x : x[x.find('講師')+3:x.find('\r\n●授業日')])


In [167]:
# 授業の様子(講師より)だけのカラムを作る
report_df['授業の様子'] = report_df['本文'].apply(lambda x : x[x.find('授業の様子(講師より)')+13:x.find('●本日')])
# 必要な列だけのdfにする
to_souma_df=report_df.loc[:, ['日付','時間', '講師','授業の様子']]
#文字コードをutf8に統一する為に、encoding="utf8"というオプションを記載
to_souma_df.to_csv("to_souma.csv",index=None,encoding="utf8")

/var/folders/x4/8hbz4flj6_bg96s9y3b69rb80000gn/T/ipykernel_64230/1369451925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_df['授業の様子'] = report_df['本文'].apply(lambda x : x[x.find('授業の様子(講師より)')+13:x.find('●本日')])
